In [1]:
import json

def get_af3_data(tag, dir):
    base = f'{dir}/{tag}'
    model_path = f'{base}/{tag}_model.cif'
    confid_path = f'{base}/{tag}_summary_confidences.json'
    with open(confid_path, 'r') as f:
        confid_dict = json.load(f)
    iptm = confid_dict['iptm']
    ptm = confid_dict['ptm']
    score = confid_dict['ranking_score']
    return {
        'path': model_path,
        'iptm': iptm,
        'ptm': ptm,
        'score': score,
    }

def get_boltz_data(tag, dir):
    base = f'{dir}/{tag}'
    model_path = f'{base}/{tag}_model_0.cif'
    confid_path = f'{base}/confidence_{tag}_model_0.json'
    affinity_path = f'{base}/affinity_{tag}.json'
    with open(confid_path, 'r') as f:
        confid_dict = json.load(f)
    score = confid_dict['confidence_score']
    iptm = confid_dict['iptm']
    ptm = confid_dict['ptm']
    with open(affinity_path, 'r') as f:
        affinity_dict = json.load(f)
    affinity_ic50 = affinity_dict['affinity_pred_value']
    affinity_bind = affinity_dict['affinity_probability_binary']
    return {
        'path': model_path,
        'iptm': iptm,
        'ptm': ptm,
        'score': score,
        'affinity_ic50': affinity_ic50,
        'affinity_bind': affinity_bind,
    }

In [2]:
import pandas as pd

def process_af3_data(df, af3_dir, name='af3', col=('lmpnn', 'tag')):
    df['af3_data'] = df[col].apply(lambda x: get_af3_data(x, af3_dir))
    df_exp = pd.json_normalize(df['af3_data']).reset_index(drop=True)
    df_exp.columns = pd.MultiIndex.from_product([[name], df_exp.columns])
    df = pd.concat([df.drop(columns=['af3_data']).reset_index(drop=True), df_exp], axis=1)
    return df

def process_boltz_data(df, boltz_dir, name='boltz', col=('lmpnn', 'tag')):
    df['boltz_data'] = df[col].apply(lambda x: get_boltz_data(x, boltz_dir))
    df_exp = pd.json_normalize(df['boltz_data']).reset_index(drop=True)
    df_exp.columns = pd.MultiIndex.from_product([[name], df_exp.columns])
    df = pd.concat([df.drop(columns=['boltz_data']).reset_index(drop=True), df_exp], axis=1)
    df.columns = pd.MultiIndex.from_tuples([
        col if isinstance(col, tuple) else (col, '')
        for col in df.columns
    ])
    return df

In [3]:
import pandas as pd

path = '../4_rscore_filter/lmpnn_filt_pht.parquet'
df = pd.read_parquet(path)

df = process_af3_data(df, '../5_af3/output', name='af3', col=('lmpnn', 'tag'))
df = process_boltz_data(df, '../6_boltz/output/boltz_results_0/predictions', name='boltz', col=('lmpnn', 'tag'))

df.to_parquet('lmpnn_filt_data_added.parquet')
df

/tmp/ipykernel_2181868/2407770881.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = pd.concat([df.drop(columns=['boltz_data']).reset_index(drop=True), df_exp], axis=1)


diffusion                           lmpnn             \
                      id     batch                   tag        ddg   
0    result_7_packed_3_1  pht_demo   result_7_packed_3_1 -27.749462   
1   result_61_packed_8_1  pht_demo  result_61_packed_8_1 -26.640675   
2    result_7_packed_8_1  pht_demo   result_7_packed_8_1 -25.789537   
3    result_7_packed_2_1  pht_demo   result_7_packed_2_1 -25.547806   
4   result_16_packed_4_1  pht_demo  result_16_packed_4_1 -25.318375   
5   result_16_packed_7_1  pht_demo  result_16_packed_7_1 -24.987370   
6    result_7_packed_4_1  pht_demo   result_7_packed_4_1 -23.823254   
7   result_29_packed_7_1  pht_demo  result_29_packed_7_1 -22.682110   
8   result_59_packed_1_1  pht_demo  result_59_packed_1_1 -22.405994   
9   result_59_packed_6_1  pht_demo  result_59_packed_6_1 -21.156240   
10   result_7_packed_1_1  pht_demo   result_7_packed_1_1 -21.103624   
11  result_24_packed_8_1  pht_demo  result_24_packed_8_1 -20.956982   
12  result_24_packed_7_1  pht_demo  result_24_packed_7_1 -20.956400   
13  result_59_packed_3_1  pht_demo  result_59_packed_3_1 -20.733452   
14  result_55_packed_1_1  pht_demo  result_55_packed_1_1 -20.355864   
15  result_59_packed_8_1  pht_demo  result_59_packed_8_1 -20.324280   
16  result_24_packed_3_1  pht_demo  result_24_packed_3_1 -20.292067   
17  result_24_packed_6_1  pht_demo  result_24_packed_6_1 -20.236935   
18  result_59_packed_7_1  pht_demo  result_59_packed_7_1 -20.159588   
19  result_55_packed_2_1  pht_demo  result_55_packed_2_1 -19.940771   
20  result_24_packed_4_1  pht_demo  result_24_packed_4_1 -19.784569   
21  result_24_packed_5_1  pht_demo  result_24_packed_5_1 -19.718063   
22  result_10_packed_4_1  pht_demo  result_10_packed_4_1 -19.292791   
23  result_29_packed_3_1  pht_demo  result_29_packed_3_1 -19.285507   
24  result_55_packed_4_1  pht_demo  result_55_packed_4_1 -19.126881   
25  result_24_packed_1_1  pht_demo  result_24_packed_1_1 -18.947884   
26  result_55_packed_5_1  pht_demo  result_55_packed_5_1 -18.944952   
27  result_24_packed_2_1  pht_demo  result_24_packed_2_1 -18.919436   
28  result_59_packed_2_1  pht_demo  result_59_packed_2_1 -18.819977   
29  result_59_packed_5_1  pht_demo  result_59_packed_5_1 -18.560551   
30  result_16_packed_2_1  pht_demo  result_16_packed_2_1 -18.512140   
31  result_55_packed_7_1  pht_demo  result_55_packed_7_1 -18.505692   
32   result_4_packed_3_1  pht_demo   result_4_packed_3_1 -18.452574   
33  result_15_packed_7_1  pht_demo  result_15_packed_7_1 -18.338469   
34  result_29_packed_6_1  pht_demo  result_29_packed_6_1 -18.066357   
35   result_5_packed_6_1  pht_demo   result_5_packed_6_1 -18.049738   

                                           \
        fa_rep res_totalscore  totalscore   
0    95.805969      -1.583578 -199.530838   
1   104.494186      -1.441779 -181.664185   
2    98.276131      -1.570810 -197.922104   
3   102.521637      -1.610180 -202.882706   
4    91.409203      -1.448796 -160.816406   
5    88.052048      -1.478883 -164.156021   
6   103.836670      -1.429071 -180.062973   
7    75.388725      -1.675448 -177.597519   
8    70.000290      -1.624849 -180.358276   
9    66.255737      -1.786232 -198.271744   
10   97.976486      -1.563029 -196.941681   
11   73.818649      -1.721328 -194.510040   
12   75.090805      -1.810861 -204.627274   
13   66.947762      -1.525246 -169.302292   
14   69.711243      -1.708151 -182.772202   
15   65.809105      -1.677221 -186.171524   
16   68.647247      -1.709947 -193.223969   
17   76.706245      -1.489178 -168.277100   
18   69.610352      -1.726851 -191.680466   
19   66.307579      -1.638223 -175.289825   
20   75.684486      -1.614937 -182.487900   
21   73.074364      -1.775147 -200.591599   
22   83.306686      -1.661632 -177.794617   
23   81.779648      -1.521618 -161.291504   
24   67.392075      -1.756452 -187.940399   
25   74.767509      -1.562683 -176.583221   
26   71.573624      -1.511618 -161.743149   
27   75.802